### TODO: ###
- Implement additional bonus score features:
    - Full clear
    - Combos
- Select NN type

In [1]:
import numpy as np

from collections import deque

import matplotlib.pyplot as plt
%matplotlib inline

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

# Gym
import gym
# import gym_pygame

# Hugging Face Hub
# from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.
# import imageio

In [2]:
env_id = "gym_tetris:tetris_rl"
env = gym.make(env_id)
eval_env = gym.make(env_id)
s_size = env.observation_space.shape
a_size = env.action_space.n

/home/ai/.local/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:20: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
/home/ai/.local/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:25: UserWarning: WARN: It seems a Box observation space is an image but the upper and lower bounds are not in [0, 255]. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  logger.warn(


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample()) # Get a random observation

print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample()) # Take a random action

_____OBSERVATION SPACE_____ 

The State Space is:  (1, 29, 10)
Sample observation [[[1.3823761  0.55758595 1.1091311  0.41747603 0.51495945 1.3864182
   1.7224386  1.073062   0.44632304 1.8973005 ]
  [0.7309322  1.3727808  1.5753471  1.1433415  1.6789643  0.5521697
   1.3483944  0.77722937 1.7047476  0.50821936]
  [1.0808114  1.3938103  0.01558153 0.165352   1.6665164  0.90415347
   0.95690686 1.058503   1.146038   0.43344957]
  [0.19206783 0.08784577 0.38814518 0.0799054  0.7228692  1.5988367
   1.1009266  0.6234679  1.288257   0.42813107]
  [0.8626415  1.7106012  1.5913675  1.6740184  1.5093381  1.8657327
   0.25874957 1.9217572  0.2686025  0.60886234]
  [0.9345636  0.6329479  0.81135106 0.87246037 1.9168451  0.2734837
   0.8348335  1.439792   1.0792379  1.2434831 ]
  [1.5764691  1.563034   1.4931544  0.02460025 1.1581335  0.6180945
   1.9023187  1.9692788  0.08114924 0.16439523]
  [1.9224085  1.2069038  0.19290449 1.7726233  1.0573136  0.5971366
   1.9962649  0.49996603 1.431833   1

In [5]:
print(env.observation_space.shape)

(1, 29, 10)


In [6]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size, h2_size):
        super(Policy, self).__init__()

        # Convolutional Network
        # self.conv = nn.Sequential(nn.Conv2d(s_size[0], h_size, 4), nn.ReLU(),
        #     nn.Conv2d(h_size, h_size, kernel_size=3, stride=1),nn.ReLU())
        # # print(self.conv(torch.zeros(1,*s_size,4)).size())
        # self.fc = nn.Sequential(
        #     nn.Linear(int(np.prod(self.conv(torch.zeros(1,*s_size)).size())), h2_size),
        #     nn.ReLU(),
        #     nn.Linear(h2_size, a_size)
        # )

        # Linear Network
        self.fc1 = nn.Linear(np.prod(s_size), h_size)
        self.fc2 = nn.Linear(h_size, h_size)
        self.fc3 = nn.Linear(h_size, a_size)

        

    def forward(self, x):
        # Convlutional Network
        # x = self.conv(x).view(x.size()[0], -1)
        # x = self.fc(x)

        # Linear Network
        x = np.reshape(x,(1,-1))
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)

        # print(np.shape(x))
        # We output the softmax
        return F.softmax(x, dim=1)

    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

In [7]:
def reinforce(policy, optimizer, n_training_episodes, max_t, gamma, print_every):
    # Help us to calculate the score during the training
    scores_deque = deque(maxlen=100)
    scores = []
    # Line 3 of pseudocode
    for i_episode in range(1, n_training_episodes+1):
        saved_log_probs = []
        rewards = []
        state = env.reset()
        # Line 4 of pseudocode
        for t in range(max_t):
            action, log_prob = policy.act(state) # Code Here: get the action
            saved_log_probs.append(log_prob)
            state, reward, done, _ = env.step(action)
            rewards.append(reward)
            if done:
                break
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))

        # Line 6 of pseudocode: calculate the return
        returns = deque(maxlen=max_t)
        n_steps = len(rewards)

        # Compute the discounted returns at each timestep,
        # as the sum of the gamma-discounted return at time t (G_t) + the reward at time t

        ## We compute this starting from the last timestep to the first, to avoid redundant computations

        ## appendleft() function of queues appends to the position 0
        ## We use deque instead of lists to reduce the time complexity

        for t in range(n_steps)[::-1]:
            disc_return_t = (returns[0] if len(returns)>0 else 0)
            returns.appendleft(gamma*disc_return_t + rewards[t]) # Code Here: complete here

        ## standardization for training stability
        eps = np.finfo(np.float32).eps.item()

        ## eps is added to the standard deviation of the returns to avoid numerical instabilities
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + eps)

        # Line 7:
        policy_loss = []
        for log_prob, disc_return in zip(saved_log_probs, returns):
            policy_loss.append(-log_prob * disc_return)
        policy_loss = torch.cat(policy_loss).sum()

        # Line 8: PyTorch prefers gradient descent
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()

        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))

    return scores

In [8]:
hyperparameters = {
    "h_size": 64,
    "h2_size": 512,
    "n_training_episodes": 10000,
    "n_evaluation_episodes": 10,
    "max_t": 10000,
    "gamma": 0.99,
    "lr": 1e-4,
    "env_id": env_id,
    "state_space": s_size,
    "action_space": a_size,
}

In [9]:
# Create policy and place it to the device
torch.manual_seed(50) # Don't change this
tetris_policy = Policy(hyperparameters["state_space"], hyperparameters["action_space"], hyperparameters["h_size"], hyperparameters["h2_size"]).to(device)
tetris_optimizer = optim.Adam(tetris_policy.parameters(), lr=hyperparameters["lr"])

In [11]:
scores = reinforce(tetris_policy,
                   tetris_optimizer,
                   hyperparameters["n_training_episodes"],
                   hyperparameters["max_t"],
                   hyperparameters["gamma"],
                   100)

torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Si

RuntimeError: zero-dimensional tensor (at position 0) cannot be concatenated